<a href="https://colab.research.google.com/github/vanilsongomes/Pratical/blob/master/MARAJOII_PROPHET_COVID19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FACULDADE DE MATEMÁTICA - UFPA/CUMB**
## **PREVISÃO DO COVID-19 NO MARAJÓ**

### <center>Analysis of the COVID-19 Pandemic Machine Learning Technics for Forecasting</center>

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
import re
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows',1000, 'display.max_columns',50)
from collections import Counter

#######################PROPHET PREDICTION #######################

from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')

##################################################################
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df_total_brazil ="https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv"
##df_total_brazil ="https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv"
df_brasil = pd.read_csv(df_total_brazil)
df_brasil.head()

,epi_week,date,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source
0,9,2020-02-25,Brazil,SP,São Paulo/SP,3550308,35016.0,São Paulo,0,0,1,1,0.0,0.00816,0.0,SES
1,9,2020-02-25,Brazil,TOTAL,TOTAL,0,NaN,NaN,0,0,1,1,0.0,0.00048,0.0,NaN
2,9,2020-02-26,Brazil,SP,São Paulo/SP,3550308,35016.0,São Paulo,0,0,0,1,0.0,0.00816,0.0,SES
3,9,2020-02-26,Brazil,TOTAL,TOTAL,0,NaN,NaN,0,0,0,1,0.0,0.00048,0.0,NaN
4,9,2020-02-27,Brazil,SP,São Paulo/SP,3550308,35016.0,São Paulo,0,0,0,1,0.0,0.00816,0.0,SES


In [3]:
marajo_PA = df_brasil[df_brasil['name_RegiaoDeSaude'] == 'Marajó II']
marajo_PA.head(2)

,epi_week,date,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source
14847,16,2020-04-15,Brazil,PA,Curralinho/PA,1502806,15014.0,Marajó II,0,0,1,1,0.0,2.90293,0.0,SES
16047,16,2020-04-16,Brazil,PA,Curralinho/PA,1502806,15014.0,Marajó II,0,0,0,1,0.0,2.90293,0.0,SES


In [4]:
#Till 15th April Cases in Marajo
df1= marajo_PA[marajo_PA['name_RegiaoDeSaude']=='Marajó II']
fig = px.bar(df1, x='city', y='totalCases', color='totalCases', height=600)
fig.update_layout(
    title='Till 28th April Total Cases in Marajó II')
fig.show()


In [5]:
#Daily Deaths in Italy
carona_data = df1.groupby(['date'])['deaths'].sum().reset_index().sort_values('deaths',ascending = True)
carona_data['Daily Deaths'] = carona_data['deaths'].sub(carona_data['deaths'].shift())
carona_data['Daily Deaths'].iloc[0] = carona_data['deaths'].iloc[0]
carona_data['Daily Deaths'] = carona_data['Daily Deaths'].astype(int)
fig = px.bar(carona_data, y='Daily Deaths', x='date',hover_data =['Daily Deaths'], color='Daily Deaths', height=500)
fig.update_layout(
    title='DAILY DEATHS IN MARAJÓ II')
fig.show()

In [6]:
#Pie chart visualization of states effected by caronavirus
fig = px.pie(df1, values='totalCases', names='city')
fig.update_layout(
    title='Pie chart visualization of city effected by caronavirus')
fig.show()

In [7]:
#Forecasting of Total Cases for Next 30 Days
df = df1.groupby('date')['totalCases'].sum().reset_index()
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Predicted Cases',marker_color='Black'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',name='Actual Cases',marker_color='red'))
fig.update_layout(
    title='Forecasting of Total Cases in Marajó I for Next 30 Days')
fig.show()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [8]:
#Forecasting of Deaths for Next 30 Days
df = df1.groupby('date')['deaths'].sum().reset_index()
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=30)
forecast = m.predict(future)
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Predicted Cases',marker_color='red'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['y'],
                    mode='lines+markers',name='Actual Cases',marker_color='blue'))
fig.update_layout(
    title='Forecasting of Deaths in MARAJÓ II for Next 30 Days')
fig.show()


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


### CASO METROPOLITANA I

In [9]:
df_PA = df_brasil[df_brasil['state'] == 'PA']
df_PA.head(2)

,epi_week,date,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source
323,12,2020-03-18,Brazil,PA,Belém/PA,1501402,15006.0,Metropolitana I,0,0,1,1,0.0,0.06699,0.0,SES
401,12,2020-03-19,Brazil,PA,Belém/PA,1501402,15006.0,Metropolitana I,0,0,0,1,0.0,0.06699,0.0,SES


In [10]:
df_metropolitana1_PA = df_PA[df_PA['name_RegiaoDeSaude'] == 'Metropolitana I']
df_metropolitana1_PA .head(2)

,epi_week,date,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,newDeaths,deaths,newCases,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source
323,12,2020-03-18,Brazil,PA,Belém/PA,1501402,15006.0,Metropolitana I,0,0,1,1,0.0,0.06699,0.0,SES
401,12,2020-03-19,Brazil,PA,Belém/PA,1501402,15006.0,Metropolitana I,0,0,0,1,0.0,0.06699,0.0,SES


In [11]:
#Pie chart visualization of states effected by caronavirus
fig = px.pie(df_metropolitana1_PA, values='totalCases', names='city')
fig.update_layout(
    title='Pie chart visualization of city effected by caronavirus')
fig.show()

### Comparar Marajó I vs Metropolitana I

In [12]:
carona_data_Metropolitana1 = df_metropolitana1_PA.groupby(['date'])['totalCases'].sum().reset_index().sort_values('totalCases',ascending = True)
carona_data_Marajo =df1.groupby(['date'])['totalCases'].sum().reset_index().sort_values('date',ascending = True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=carona_data_Metropolitana1['date'], y=carona_data_Metropolitana1['totalCases'],
                    mode='lines+markers',name='Metropolitana I'))
fig.add_trace(go.Scatter(x=carona_data_Marajo['date'], y=carona_data_Marajo['totalCases'],
                    mode='lines+markers',name='Marajo II'))
fig.update_layout(title_text='Total Confirmed Cases METROPOLITANA I vs MARAJÓ II',plot_bgcolor='rgb(225,230,255)')
fig.show()

# Referências
[Scikit-Learning](https://scikit-learn.org/)

[Time series prediction using Prophet in Python](https://towardsdatascience.com/time-series-prediction-using-prophet-in-python-35d65f626236)

[Observatório de Dados :: COVID-19 no Brasil  - UFPA](http://ccsl.ufpa.br/covid-19/)

[dataset: Número de casos confirmados de COVID-19 no Brasil](https://covid19br.wcota.me/)